# Rappler 2022 Scitech Database Maker

In [66]:
import numpy as np
import pandas as pd
import requests
import bs4
import re
import datetime
import time
import sqlite3
import db_caller
from tqdm.notebook import tqdm

## Create + Connect to SQL Database

In [67]:
conn = db_caller.connect("LR2.db")

## 2022 Rappler Section/Article Link Data

### Webscrape Section Links

In [2]:
# Request main page
proxies = {'http': 'http://206.189.157.23'}
resp = requests.get('http://www.rappler.com', proxies=proxies)
soup = bs4.BeautifulSoup(resp.text)

# Get section links
topic_li = soup.select('nav.burger-nav > div[id^="submenu-panel-"][data-depth="1"]')[0].select('li')
topics = dict([(i.text.casefold(), i.select_one('a')['href']) for i in topic_li][1:-1])

# Get subsection links + topic-subtopic dictionary
subtopic_li = [i.select('li') for i in soup.select('nav.burger-nav > div[id^="submenu-panel-"][data-depth="2"]')][1:]
topic_tree = {}
subtopics = []
for topic, links in zip(topics, subtopic_li):
    branch = []
    for link in links:
        if not bool(re.match(r'^see all', link.text.casefold())):
            subtopics.append((link.text.casefold(), link.select_one('a')['href']))
            branch.append(link.text.casefold())
    topic_tree[topic] = branch
subtopics = dict(subtopics)

### Webscrape Article Links

In [51]:
# Get articles
articles = []

for topic in tqdm(topic_tree):
    print(f'On topic page: {topic}')
    
    for subtopic in tqdm(topic_tree[topic]):
        print(f'    On subtopic page: {subtopic}')
        
        # Request subtopic page
        subtopic_page = subtopics[subtopic]
        soup = bs4.BeautifulSoup(requests.get(subtopic_page, proxies=proxies).text)

        while True:
            # Scrape info per article in subtopic page
            for i in soup.select('article'):
                article_topic = i.select_one('div > a').text.strip().casefold()
                article_title = i.select_one('h2 > a').text.strip().casefold()
                article_link = i.select_one('h2 > a')['href']
                article_date = i.select_one('time')['datetime']
                retrieval_date = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S+00:00')
                # Append article info if published in 2022
                if article_date[:4] == '2022':
                    stop_2022 = False
                    articles.append([topic, subtopic, article_title, article_topic, article_link, article_date, retrieval_date])
                else:
                    stop_2022 = True

            # If articles are no longer published in 2022, move to next subtopic
            if stop_2022:
                break
            # If there is still a next page, move to next page
            elif soup.select_one('div.pagination'):
                next_page = soup.select_one('div.pagination a')['href']
                soup = bs4.BeautifulSoup(requests.get(next_page, proxies=proxies).text)
                time.sleep(np.random.choice(np.arange(1,10)))
                print(f'        {next_page}')
            else:
                break

  0%|          | 0/14 [00:00<?, ?it/s]

On topic page: video


  0%|          | 0/12 [00:00<?, ?it/s]

    On subtopic page: hold the line
    On subtopic page: video editorial
        https://www.rappler.com/section/video/editorial/page/2/
        https://www.rappler.com/section/video/editorial/page/3/
        https://www.rappler.com/section/video/editorial/page/4/
    On subtopic page: act one
    On subtopic page: basagan ng trip
    On subtopic page: daily wrap
        https://www.rappler.com/section/video/daily-wrap/page/2/
        https://www.rappler.com/section/video/daily-wrap/page/3/
        https://www.rappler.com/section/video/daily-wrap/page/4/
        https://www.rappler.com/section/video/daily-wrap/page/5/
        https://www.rappler.com/section/video/daily-wrap/page/6/
        https://www.rappler.com/section/video/daily-wrap/page/7/
        https://www.rappler.com/section/video/daily-wrap/page/8/
        https://www.rappler.com/section/video/daily-wrap/page/9/
        https://www.rappler.com/section/video/daily-wrap/page/10/
        https://www.rappler.com/section/video/d

  0%|          | 0/14 [00:00<?, ?it/s]

...

    On subtopic page: advocacies
        https://www.rappler.com/section/moveph/advocacies/page/2/
        https://www.rappler.com/section/moveph/advocacies/page/3/
        https://www.rappler.com/section/moveph/advocacies/page/4/
        https://www.rappler.com/section/moveph/advocacies/page/5/
        https://www.rappler.com/section/moveph/advocacies/page/6/
        https://www.rappler.com/section/moveph/advocacies/page/7/
    On subtopic page: agos
    On subtopic page: partners
On topic page: science


  0%|          | 0/4 [00:00<?, ?it/s]

    On subtopic page: discoveries & inventions
    On subtopic page: earth & space
        https://www.rappler.com/section/science/earth-space/page/2/
        https://www.rappler.com/section/science/earth-space/page/3/
        https://www.rappler.com/section/science/earth-space/page/4/
        https://www.rappler.com/section/science/earth-space/page/5/
        https://www.rappler.com/section/science/earth-space/page/6/
    On subtopic page: life & health
        https://www.rappler.com/section/science/life-health/page/2/
        https://www.rappler.com/section/science/life-health/page/3/
        https://www.rappler.com/section/science/life-health/page/4/
        https://www.rappler.com/section/science/life-health/page/5/
        https://www.rappler.com/section/science/life-health/page/6/
        https://www.rappler.com/section/science/life-health/page/7/
        https://www.rappler.com/section/science/life-health/page/8/
        https://www.rappler.com/section/science/life-health/page/

  0%|          | 0/4 [00:00<?, ?it/s]

    On subtopic page: climate change
        https://www.rappler.com/section/environment/climate-change/page/2/
        https://www.rappler.com/section/environment/climate-change/page/3/
        https://www.rappler.com/section/environment/climate-change/page/4/
        https://www.rappler.com/section/environment/climate-change/page/5/
        https://www.rappler.com/section/environment/climate-change/page/6/
        https://www.rappler.com/section/environment/climate-change/page/7/
        https://www.rappler.com/section/environment/climate-change/page/8/
        https://www.rappler.com/section/environment/climate-change/page/9/
        https://www.rappler.com/section/environment/climate-change/page/10/
        https://www.rappler.com/section/environment/climate-change/page/11/
        https://www.rappler.com/section/environment/climate-change/page/12/
    On subtopic page: disasters in the philippines
    On subtopic page: nature
        https://www.rappler.com/section/environment/nat

### Load Article Links Table

In [74]:
cols = ['Page Topic', 'Page Subtopic', 'Article Title', 'Article Topic', 'Article Link', 'Article Date', 'Retrieval Date']
articles_df = pd.DataFrame(articles, columns=cols)
articles_df = articles_df.drop_duplicates(['Article Title'])
articles_df = articles_df[articles_df['Article Topic'] != 'rappler+ exclusives']
print(articles_df.shape)
articles_df.head()

(19942, 7)


,Page Topic,Page Subtopic,Article Title,Article Topic,Article Link,Article Date,Retrieval Date
1,video,hold the line,#holdtheline: maria ressa talks to senator ris...,hold the line,https://www.rappler.com/video/hold-the-line-ma...,2022-06-07T02:00:00+00:00,2022-11-21T07:00:46+00:00
2,video,hold the line,full video: maria ressa talks to frances haugen,exclusive events,https://www.rappler.com/plus-membership-progra...,2022-03-25T05:52:25+00:00,2022-11-21T07:00:46+00:00
4,video,video editorial,[video editorial] huwag kalimutan si percy lapid,editorials,https://www.rappler.com/voices/editorials/vide...,2022-10-29T04:16:55+00:00,2022-11-21T07:00:46+00:00
5,video,video editorial,[video editorial] sandaang araw ni marcos: ‘de...,editorials,https://www.rappler.com/voices/editorials/vide...,2022-10-14T09:40:43+00:00,2022-11-21T07:00:46+00:00
6,video,video editorial,[video editorial] bakit ‘fail’ kapag influence...,editorials,https://www.rappler.com/voices/editorials/vide...,2022-10-07T11:02:40+00:00,2022-11-21T07:00:46+00:00


In [75]:
try:
    articles_df.to_sql('rappler_2022', conn, if_exists='fail', index=False)
except ValueError:
    print('Already in database.')

Already in database.


## 2022 Rappler Scitech Article Data

In [130]:
scitech_df = articles_df[(articles_df['Page Topic'] == 'technology') | (articles_df['Page Topic'] == 'science')]
scitech_links = scitech_df['Article Link'].to_numpy()
scitech_df = scitech_df[['Page Topic', 'Article Topic', 'Article Title']]
print(scitech_df.shape)
scitech_df.head()

(465, 3)


,Page Topic,Article Topic,Article Title
22732,technology,social media,google approves trump’s truth social for play ...
22733,technology,technology,"password manager lastpass reports breach, says..."
22734,technology,technology,tiktok refutes researcher’s claims that in-app...
22735,technology,apps,messaging app telegram to launch paid subscrip...
22736,technology,technology,grabmaps seeks to provide more accurate mappin...


### Webscrape Scitech Article Contents

In [132]:
# Get science and technology article summary and contents
scitech_summary = []
scitech_content = []

for link in tqdm(scitech_links):
    soup = bs4.BeautifulSoup(requests.get(link, proxies=proxies).text)
    summary = soup.select_one('div.post-single__summary').text.strip().casefold()
    summary = re.sub(r'\s', r' ', summary)
    scitech_summary.append(summary)
    
    content = soup.select_one('div.post-single__content').text.strip().casefold()
    content = re.sub(r'\s', r' ', content)
    scitech_content.append(content)

    time.sleep(np.random.choice(np.arange(1, 8)))

  0%|          | 0/465 [00:00<?, ?it/s]

### Load Scitech Article Contents

In [138]:
scitech_df['Summary'] = scitech_summary
scitech_df['Content'] = scitech_content
print(scitech_df.shape)
scitech_df.head()

(465, 5)


,Page Topic,Article Topic,Article Title,Summary,Content
22732,technology,social media,google approves trump’s truth social for play ...,"trump media & technology group, which operates...",alphabet inc’s google has approved former us p...
22733,technology,technology,"password manager lastpass reports breach, says...",lastpass ceo karim toubba says an investigatio...,"washington, dc, usa – popular digital password..."
22734,technology,technology,tiktok refutes researcher’s claims that in-app...,"security researcher felix krause says tiktok, ...",play video a new analysis revealed that ...
22735,technology,apps,messaging app telegram to launch paid subscrip...,'the only way to let our most demanding fans g...,messaging app telegram will launch a paid subs...
22736,technology,technology,grabmaps seeks to provide more accurate mappin...,grab is completing its move to use its own map...,"manila, philippines – grab on wednesday, june ..."


In [140]:
try:
    scitech_df.to_sql('rappler_scitech', conn, if_exists='fail', index=False)
except ValueError:
    print('Already in database.')

Already in database.
